In [ ]:
# | default_exp _helpers.api_docs_helper

In [ ]:
# | export

from typing import *
import types
from pathlib import Path
from inspect import isfunction, isclass, getmembers, getsourcefile, isroutine

import griffe
from nbdev.config import get_config

In [ ]:
from subprocess import CalledProcessError
import unittest.mock
from contextlib import contextmanager

from nbdev.doclinks import NbdevLookup

from nbdev_mkdocs.mkdocs import prepare

/Users/harishm/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
# | export


def _get_symbol_filepath(symbol: Union[types.FunctionType, Type[Any]]) -> Path:
    config = get_config()
    filepath = getsourcefile(symbol)
    return Path(filepath).relative_to( # type: ignore
        filepath.split(f'{config["lib_name"].replace("-", "_")}/')[0] # type: ignore
    )

In [ ]:
# actual = _get_symbol_filepath(prepare)
# expected = Path("nbdev_mkdocs/mkdocs.py")
# print(actual)

# assert actual == expected

In [ ]:
# | export


def _get_annotated_symbol_definition(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    try:
        module = f"{symbol.__module__}.{symbol.__qualname__}"
        parsed_module = griffe.load(module)
        if "raise NotImplementedError()" in parsed_module.source:
            raise KeyError
        return f"\n\n::: {module}\n"
    except KeyError as e:
        patched_symbol_path = _get_symbol_filepath(symbol)
        return f"\n\n::: {str(patched_symbol_path.parent)}.{str(patched_symbol_path.stem)}.{symbol.__name__}\n"

In [ ]:
_get_annotated_symbol_definition_mock_value = f'::: nbdev_mkdocs.mkdocs.prepare'

@contextmanager
def mock_get_annotated_symbol_definition():
    with unittest.mock.patch('__main__._get_annotated_symbol_definition') as mock_get_annotated_symbol_definition:
        mock_get_annotated_symbol_definition.return_value = _get_annotated_symbol_definition_mock_value
        yield


In [ ]:
with mock_get_annotated_symbol_definition():
    actual = _get_annotated_symbol_definition(prepare)

expected = """::: nbdev_mkdocs.mkdocs.prepare"""
display(actual)
assert actual == expected

'::: nbdev_mkdocs.mkdocs.prepare'

In [ ]:
# | export


def _get_attributes_to_exclude_in_docstring(
    symbol: Union[types.FunctionType, Type[Any]]
) -> str:
    members_list = [
        f'"!^{a}$"'
        for a in dir(symbol)
        if callable(getattr(symbol, a)) and (not a.startswith("__") or a == "__init__")
    ]
    return f"""    options:
      filters: [{", ".join(members_list)}]"""

In [ ]:
class Car:
    SOME_ATTRIBUTE = "Some class attribute"

    def __init__(self, make: str, model: str, year: int, color: str):
        self.make = make
        self.model = model
        self.year = year
        self.color = color
        self.is_running = False

    def start(self):
        self.is_running = True

    def stop(self):
        self.is_running = False

    def drive(
        self,
        distance: float,
        speed: Optional[float] = None,
        passengers: Optional[int] = None,
    ) -> float:
        if not self.is_running:
            raise ValueError("Cannot drive a stopped car.")

        if speed:
            print(f"Driving at {speed} km/h.")

        if passengers:
            print(f"Driving with {passengers} passengers.")

        return distance
        
    def patched_method_in_same_file(self, s: str) -> str:
        raise NotImplementedError()

    @staticmethod
    def i_am_a_static_method(name: str) -> str:
        if not isinstance(name, str):
            raise ValueError("I will accept only string")
        return f"Hello, {name}"
    
    @classmethod
    def i_am_a_class_method(cls):
        return f"Nothing"
        
    
        
actual = _get_attributes_to_exclude_in_docstring(Car)
expected = """    options:
      filters: ["!^__init__$", "!^drive$", "!^i_am_a_class_method$", "!^i_am_a_static_method$", "!^patched_method_in_same_file$", "!^start$", "!^stop$"]"""
display(actual)
assert actual == expected

'    options:\n      filters: ["!^__init__$", "!^drive$", "!^i_am_a_class_method$", "!^i_am_a_static_method$", "!^patched_method_in_same_file$", "!^start$", "!^stop$"]'

In [ ]:
# | export



def get_formatted_docstring_for_symbol(
    symbol: Union[types.FunctionType, Type[Any]]
) -> str:
    """Recursively parses and get formatted docstring of a symbol.

    Args:
        symbol: A Python class or function object to parse the docstring for.

    Returns:
        A formatted docstring of the symbol and its members.

    """

    def traverse(symbol: Union[types.FunctionType, Type[Any]], contents: str) -> str:
        """Recursively traverse the members of a symbol and append their docstrings to the provided contents string.

        Args:
            symbol: A Python class or function object to parse the docstring for.
            contents: The current formatted docstrings.

        Returns:
            The updated formatted docstrings.

        """
        for x, y in getmembers(symbol):
            if not x.startswith("_") or x == "__init__":
                if isfunction(y) and y.__doc__ is not None:
                    contents += f"{_get_annotated_symbol_definition(y)}\n\n"
                elif isclass(y) and not x.startswith("__") and y.__doc__ is not None:
                    contents += "\n" + _get_attributes_to_exclude_in_docstring(y) + "\n\n"
                    contents = traverse(y, contents)
        return contents

    if symbol.__doc__ is None:
        return ""
    
    contents = _get_annotated_symbol_definition(symbol)
    if isclass(symbol):
        contents += _get_attributes_to_exclude_in_docstring(symbol) + "\n\n"
        contents = traverse(symbol, contents)
    return contents

In [ ]:
class Car:
    """A class representing a car.

    Attributes:
        make: The make of the car.
        model: The model of the car.
        year: The year the car was made.
        color: The color of the car.
    """
    
    SOME_CLASS_ATTRIBUTE = "some class attribute"
    second_class_attribute = "second_class_attribute"

    def __init__(self, make: str, model: str, year: int, color: str):
        """Initialize a new car.

        Args:
            make: The make of the car.
            model: The model of the car.
            year: The year the car was made.
            color: The color of the car.
        """
        self.make = make
        self.model = model
        self.year = year
        self.color = color
        self.is_running = False

with mock_get_annotated_symbol_definition():
    actual = get_formatted_docstring_for_symbol(Car)

expected = """::: nbdev_mkdocs.mkdocs.prepare    options:
      filters: ["!^__init__$"]

::: nbdev_mkdocs.mkdocs.prepare

"""
    
display(actual)
assert actual == expected

'::: nbdev_mkdocs.mkdocs.prepare    options:\n      filters: ["!^__init__$"]\n\n::: nbdev_mkdocs.mkdocs.prepare\n\n'

In [ ]:
def fixture_function(
    arg_1: str,
    arg_2: Union[List[str], str],
    arg_3: Optional[int],
    arg_4: Optional[str] = None,
) -> str:
    """This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4

    Returns:
        The concatinated string
    """
    pass
    
get_module_source_value = """
def fixture_function(
    arg_1: str,
    arg_2: Union[List[str], str],
    arg_3: Optional[int],
    arg_4: Optional[str] = None,
) -> str:
    \"\"\"This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4

    Returns:
        The concatinated string
    \"\"\"
    pass
"""

expected = "::: nbdev_mkdocs.mkdocs.prepare"

with mock_get_annotated_symbol_definition():
    actual = get_formatted_docstring_for_symbol(fixture_function)
display(actual)
assert actual == expected

'::: nbdev_mkdocs.mkdocs.prepare'